In [1]:
import proplot as pplt
import numpy as np
from mdet_tools import meas_m, meas_mbmeds, make_obs, format_mc_res

%load_ext autoreload
%autoreload 2

In [2]:
n_stars = 20
res = meas_m(n_stars=n_stars, mask_width=2, seed=20, n_jobs=100, sym=3)

print("\nstar density: %s [num per arcmin^2]" % (n_stars / (235*0.20/60)**2))
print(format_mc_res(res))

m1 +/- merr: -78.701946 +/- 16.067155 [10^(-3), 3sigma]                        
c2 +/- cerr: -1.384618 +/- 1.667711 [10^(-5), 3sigma]                          
m2 +/- merr: 92.215545 +/- 113.061696 [10^(-3), 3sigma]                        
c1 +/- cerr: 0.618158 +/- 1.871913 [10^(-5), 3sigma]                           
m1 +/- merr: 0.650393 +/- 178.476302 [10^(-3), 3sigma]                         
c2 +/- cerr: -1.533134 +/- 1.459445 [10^(-5), 3sigma]                          
m2 +/- merr: 25.734117 +/- 149.832225 [10^(-3), 3sigma]                        
c1 +/- cerr: 0.380380 +/- 1.024445 [10^(-5), 3sigma]                           
m1 +/- merr: 12.532495 +/- 122.853359 [10^(-3), 3sigma]                        
c2 +/- cerr: -1.465834 +/- 1.676566 [10^(-5), 3sigma]                          
m2 +/- merr: -3.089141 +/- 109.744183 [10^(-3), 3sigma]                        
c1 +/- cerr: 0.487910 +/- 0.874161 [10^(-5), 3sigma]                           
m1 +/- merr: 11.735169 +/- 92.258869 [10

m2 +/- merr: -21.670640 +/- 34.087931 [10^(-3), 3sigma]                        
c1 +/- cerr: 0.124800 +/- 0.448774 [10^(-5), 3sigma]                           
m1 +/- merr: 0.487901 +/- 38.551258 [10^(-3), 3sigma]                          
c2 +/- cerr: -1.337813 +/- 0.526046 [10^(-5), 3sigma]                          
m2 +/- merr: -21.294390 +/- 32.414028 [10^(-3), 3sigma]                        
c1 +/- cerr: 0.149585 +/- 0.420654 [10^(-5), 3sigma]                           
m1 +/- merr: -1.540985 +/- 38.620568 [10^(-3), 3sigma]                         
c2 +/- cerr: -1.307532 +/- 0.517515 [10^(-5), 3sigma]                          
m2 +/- merr: -22.547928 +/- 32.300908 [10^(-3), 3sigma]                        
c1 +/- cerr: 0.163816 +/- 0.430406 [10^(-5), 3sigma]                           
m1 +/- merr: -0.993467 +/- 34.970447 [10^(-3), 3sigma]                         
c2 +/- cerr: -1.291555 +/- 0.497685 [10^(-5), 3sigma]                          
m2 +/- merr: -23.858033 +/- 32.223568 [1

In [ ]:
n_stars = 20
res = meas_m(n_stars=n_stars, mask_width=2, seed=20, n_jobs=10, sym=False)

print("\nstar density: %s [num per arcmin^2]" % (n_stars / (235*0.20/60)**2))
print(format_mc_res(res))

In [ ]:
n_stars = 20
step = 0.01
s = 10
kwargs = {"n_stars": n_stars, "seed": s}
d, mbobs = meas_mbmeds(make_obs(mcal_shear=(0, 0), **kwargs), mask_width=2, sym=3)
d1p, mbobs1p = meas_mbmeds(make_obs(mcal_shear=(step, 0), **kwargs), mask_width=2, sym=3)
d1m, mbobs1m = meas_mbmeds(make_obs(mcal_shear=(-step, 0), **kwargs), mask_width=2, sym=3)

fig, axs = pplt.subplots(nrows=1, ncols=3, sharex=3, sharey=3)
axs[0].pcolormesh(mbobs[0][0].image)
axs[0].plot(d["x"], d["y"], '.b')
axs[0].set_title("mcal noshear")

axs[1].pcolormesh(mbobs1p[0][0].image)
axs[1].plot(d1p["x"], d1p["y"], '.b')
axs[1].set_title("mcal +g1")

axs[2].pcolormesh(mbobs1m[0][0].image)
axs[2].plot(d1m["x"], d1m["y"], '.b')
axs[2].set_title("mcal -g1")

axs.set_xlim(0, mbobs[0][0].image.shape[1])
axs.set_ylim(0, mbobs[0][0].image.shape[0])

In [ ]:
d

In [ ]:
import galsim
import ngmix
from ngmix.gaussmom import GaussMom
from pizza_cutter.slice_utils.interpolate import interpolate_image_and_noise


def meas_one_im(*, g1, g2, mask_delta1, mask_delta2, rotate, symdelta=0, interp=False):
    rng = np.random.RandomState()

    obj = galsim.Exponential(half_light_radius=0.5).shear(g1=g1, g2=g2)
    psf = galsim.Gaussian(fwhm=0.9).withFlux(1e6)
    obj = galsim.Convolve([obj, psf])
    dim = 53
    cen = (dim-1)//2
    im = obj.drawImage(nx=dim, ny=dim, scale=0.2).array
    psf_im = psf.drawImage(nx=dim, ny=dim, scale=0.2).array
    jac = ngmix.DiagonalJacobian(scale=0.2, row=cen, col=cen)
    psf_obs = ngmix.Observation(
        image=psf_im,
        weight=np.ones_like(psf_im),
        jacobian=jac,
    )
    wgt = np.ones_like(im)
    if mask_delta1 is not None:
        wgt[:, cen+mask_delta1:] = 0
        if rotate:
            wgt[cen+mask_delta1+symdelta:, :] = 0
    if mask_delta2 is not None:
        wgt[cen+mask_delta2:, :] = 0
        if rotate:
            wgt[:, cen+mask_delta2+symdelta:] = 0
            
    if interp:
        nse = rng.normal(size=im.shape)
        iim, inse = interpolate_image_and_noise(
            image=im,
            noises=[nse],
            weight=wgt,
            bmask=np.zeros_like(im, dtype=np.int32),
            bad_flags=0,
            rng=rng,
            fill_isolated_with_noise=True
        )
        
    obs = ngmix.Observation(
        image=im,
        weight=wgt,
        jacobian=jac,
        psf=psf_obs
    )
    mom = GaussMom(fwhm=1.2)
    res = mom.go(obs=obs)
    
    gauss_wgt = ngmix.GMixModel(
        [0, 0, 0, 0, ngmix.moments.fwhm_to_T(1.2), 1],
        'gauss',
    )    
    cobs = obs.copy()
    cobs.image = 1.0 - wgt
    cobs.weight = np.ones_like(wgt)
    stats = gauss_wgt.get_weighted_sums(
        cobs,
        1.2 * 2,
    )    
    mfrac = stats["sums"][5] / stats["wsum"]
    return res["e"][0], res["e"][1], obs, mfrac

In [ ]:
def meas_R_mask(mask_delta1, rotate, symdelta=0, interp=False):
    e1 = []
    shear = np.linspace(0, 0.06, 50)
    for s in shear:
        e1.append(meas_one_im(
            g1=s, g2=0, mask_delta1=mask_delta1, mask_delta2=None, 
            rotate=rotate, symdelta=symdelta, interp=interp
        )[0])
    e1 = np.array(e1)
    R = (e1[1:] - e1[:-1])/(shear[1:] - shear[:-1])
    sp = (shear[1:] + shear[:-1])/2
    ds = sp[1] - sp[0]
    R0 = (
        meas_one_im(
            g1=ds, g2=0, mask_delta1=mask_delta1, mask_delta2=None, 
            rotate=rotate, symdelta=symdelta, interp=interp
        )[0] 
        - meas_one_im(
            g1=-ds, g2=0, mask_delta1=mask_delta1, mask_delta2=None, 
            rotate=rotate, symdelta=symdelta, interp=interp
        )[0]
    )/2/ds
    sp = np.concatenate([np.array([0]), sp])
    R = np.concatenate([np.array([R0]), R])
    mfrac = meas_one_im(
        g1=s, g2=0, mask_delta1=mask_delta1, mask_delta2=None, 
        rotate=rotate, symdelta=symdelta, interp=interp
    )[-1]
    
    return sp, R, mfrac

In [ ]:
def _do_plots(axs, rotate, symdelta, pixvals, interp, **kwargs):
    hdls = []
    for pval in pixvals:
        if pval is None:
            sp, R, mf = meas_R_mask(None, rotate, symdelta=symdelta)
            hdls.append(axs.plot(sp, R/R[0], label='no masking'))
        else:
            sp, R, mf = meas_R_mask(pval, rotate, symdelta=symdelta)
            hdls.append(axs.plot(
                    sp, R/R[0], 
                    label='%d pix - rotated=%s - symdelta=%d - interp=%d - mfrac=%0.3f ' % (
                        pval, rotate, symdelta, interp, mf
                    ), 
                    **kwargs
                )
            )

    return hdls

fig, axs = pplt.subplots(figsize=(8, 6))

hdls = _do_plots(axs, False, 0, [None, 2, 4, 8], False)
hdls.extend(_do_plots(axs, True, 0, [2], False, linestyle='dashed', color='purple'))
hdls.extend(_do_plots(axs, True, 1, [2], False, linestyle='dotted', color='purple'))
hdls.extend(_do_plots(axs, True, 0, [2], True, linestyle='dashed', color='cyan'))

axs.legend(hdls, loc='ul', ncols=1, frameon=False, framealpha=0)
axs.format(xlabel='true shear')
axs.format(ylabel='response(shear) / response(shear=0)')
axs.grid(False)